In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.pipeline import Pipeline

In [2]:
data=pd.read_csv("Monthly_Training_Data_1.csv",usecols=["Service Name","Month","Year","Count"])
ServiceNames = {name : idx for idx, name in enumerate(data['Service Name'].unique())}
ServiceIndex= {idx : name for idx, name in enumerate(data['Service Name'].unique())}
processed_data = data.copy()
processed_data['Service Name']=processed_data['Service Name'].map(ServiceNames)
X=processed_data.drop(["Count"],axis=1)
y=processed_data["Count"]

In [3]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((681, 3), (171, 3), (681,), (171,))

In [4]:
def predict_last_two(pred, regressor):
    lasttwo = processed_data[(processed_data['Year'] == 2018) &
         ((processed_data['Month'] == 11) | (processed_data['Month'] == 12))]
    pred = regressor.predict(lasttwo.drop('Count', axis=1))
    return mean_absolute_error(pred, lasttwo['Count'])

In [15]:
def perform_regression(regressor):
    # regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    print(f'Total MAE {mean_absolute_error(y_pred, y_test)}')
    print(f'Last Two months MAE {predict_last_two(y_pred, regressor)}')
    return mean_absolute_error(y_pred, y_test), predict_last_two(y_pred, regressor)

In [32]:
dt = DecisionTreeRegressor()
parameter = {'max_depth':[10, 50, 100, 200],'min_samples_split':[2, 4, 6, 8], 
                  'min_samples_leaf':[2,4,6,8], 'criterion': ['mse', 'friedman_mse', 'mae']
                  }

scorer = make_scorer(mean_absolute_error)

dtsearch = GridSearchCV(dt, parameter, scoring = scorer, verbose=1, cv=3, n_jobs=-1)

dtfit = dtsearch.fit(X_train, y_train)
best_fit = dtfit.best_estimator_
# best_fit.fit(X_train, y_train)
# dt_pred = best_fit.predict(X_test)

Fitting 3 folds for each of 192 candidates, totalling 576 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:    1.5s finished


In [33]:
perform_regression(best_fit)

Total MAE 19.681286549707604
Last Two months MAE 13.08695652173913


(19.681286549707604, 13.08695652173913)